<a id='initialisation'></a>
___
___
## <font color='coral'>**Appel de l'environment** </font> [▲](#sommaire)

In [7]:
%reset -f
import ipywidgets as widgets
%matplotlib widget
from ipywidgets import Layout, interact
import h5py
import numpy as np
import matplotlib.pyplot as plt
from RVutils import *

#Fourier2D()
#hdf5_to_dict()

In [9]:
plt.rcParams['axes.formatter.limits'] = (-5,6)

q05_spectra = hdf5_to_dict("data/q05_growthrate_2Dspectra.hdf5")
q15_spectra = hdf5_to_dict("data/q15_growthrate_2Dspectra.hdf5")

print(q05_spectra.keys())

NameError: name 'h5py' is not defined